In [ ]:
!pip install transformers pandas scikit-learn torch

In [5]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Step 1: Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/phishing_model_v1_after_phase1", num_labels=2
)

In [8]:
import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/phishing_chunks/phase2_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/phishing_chunks/phase2_test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


Train shape: (268128, 2)
Test shape: (67033, 2)


In [9]:
train_texts = train_df["url"].tolist()
train_labels = train_df["label"].tolist()

test_texts = test_df["url"].tolist()
test_labels = test_df["label"].tolist()


In [10]:
def batch_tokenize(texts, labels, tokenizer, batch_size=10000):
    encodings = {"input_ids": [], "attention_mask": []}
    all_labels = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        batch_enc = tokenizer(batch_texts, truncation=True, padding=True, return_tensors="pt")

        encodings["input_ids"].extend(batch_enc["input_ids"])
        encodings["attention_mask"].extend(batch_enc["attention_mask"])
        all_labels.extend(batch_labels)

    return encodings, all_labels


In [11]:
train_encodings, train_labels_final = batch_tokenize(train_texts, train_labels, tokenizer)
test_encodings, test_labels_final = batch_tokenize(test_texts, test_labels, tokenizer)


In [12]:
import torch

class PhishingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {key: self.encodings[key][idx] for key in self.encodings} | {
            "labels": torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = PhishingDataset(train_encodings, train_labels_final)
test_dataset = PhishingDataset(test_encodings, test_labels_final)

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/phishing_model_v1_after_phase1",
    num_labels=2
)


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/phishing_model_checkpoint",  # Overwrites with Phase 3
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/content/drive/MyDrive/phishing_logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
    logging_steps=50,
    report_to="none"
)

In [15]:
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

trainer.train()
trainer.evaluate()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.020500,0.027735,0.993332,0.980235,0.942476,0.960985


{'eval_loss': 0.027734674513339996,
 'eval_accuracy': 0.9933316426237823,
 'eval_precision': 0.9802350427350427,
 'eval_recall': 0.9424756034925527,
 'eval_f1': 0.9609845509295627,
 'eval_runtime': 930.932,
 'eval_samples_per_second': 72.006,
 'eval_steps_per_second': 2.25,
 'epoch': 1.0}